In [4]:
from data_preparator import DataPreparator
test = DataPreparator.load_from_pickle('pickle/train_documents.pkl')

Object loaded from pickle/train_documents.pkl


In [1]:
import os

# Initialize an empty dictionary to store the mappings
file_mapping = {}

# Walk through the folder structure
for root, dirs, files in os.walk("collections"):
    for file in files:
        # Check if the file has a .txt extension
        if file.endswith(".txt"):
            # Get the full path of the file
            file_path = os.path.join(root, file)
            
            # Extract the file name (without extension)
            file_name = os.path.splitext(file)[0]
            
            # Add the mapping to the dictionary
            file_mapping[file_name] = file_path

In [79]:
from retrieval import Retrieval
from letor import LETOR
import re

ranker_path = "lgbr_base.txt"
letor = LETOR(ranker_path)
retrieval = Retrieval()

No sentence-transformers model found with name C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [174]:
def main_pipeline_letor(query, k=100):

    clean_text = " ".join(re.findall(r"\w+", query))
    terms = retrieval.data_preparator.preprocess_text(clean_text).split(' ')

    doc_paths, _ = retrieval.retrieve_documents(terms)
    doc_ids = [doc_path.split('\\')[-1][:-4] for doc_path in doc_paths]
    dataset = retrieval.data_preparator.prepare_testing_dataset(terms, doc_paths)

    # melakukan preparation data test dan prediksi 
    X = letor.ranker.prepare_data_test(dataset)
    pred = letor.ranker.predict_ranker(X) +  X[:,-1]/100
    doc_pred_pairs = list(zip(doc_ids, pred))
    sorted_pairs = sorted(doc_pred_pairs, key=lambda x: x[1], reverse=True)
    top_k_doc_ids_letor = [pair[0] for pair in sorted_pairs]
    top_k_doc_ids_bsbi = doc_ids

    intersection = [x for x in top_k_doc_ids_letor if x in top_k_doc_ids_bsbi]
    intersection = intersection + [x for x in top_k_doc_ids_letor if x not in intersection]

    return intersection[:k]

In [180]:
result = main_pipeline_letor("cardiovascular disease", 
                                                k=50)

In [181]:
print(result)

['5468651', '5641487', '50143', '821954', '52209', '60427', '17061', '835748', '47209', '4747270', '11288', '2686161', '990', '16366', '477242', '24148', '17063', '7065630', '28660', '3443459', '49378', '16844', '22726', '2884753', '169434', '60609', '4095426', '2166', '2167', '54999', '13060', '6680388', '841697', '18221', '12957', '7379223', '5410023', '13162', '64179', '12965', '5418192', '5394632', '2155841', '5302845', '1267712', '1793583', '15550', '4658404', '5282880', '3519021']


In [182]:
# Open the file for reading
with open(file_mapping['5468651'], "r") as file:
    # Read and print the content
    content = file.read()
    print(content)

Sleep apnea is known to contribute to the development of cardiovascular disease and pulmonary hypertension. It happens because the disease increases the risk of hypertension, pulmonary vascular disease, ischemic heart disease, stroke, congestive heart failure and arrhythmias.


In [6]:
from data_preparator import DataPreparator
DataPreparator.save_to_pickle(file_mapping, "pickle/mapping_doc.pkl")

Object saved as pickle/mapping_doc.pkl


In [7]:
mapping = DataPreparator.load_from_pickle("pickle/mapping_doc.pkl")

Object loaded from pickle/mapping_doc.pkl


In [4]:
file_path

'collections\\9\\8826292.txt'

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
file_path = 'stopwords.txt'
stop_words = set(stopwords.words('english'))
with open(file_path, 'w') as file:
    for word in stop_words:
        file.write(word + '\n')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
stop_words = set(line.strip() for line in open('stopwords.txt'))

In [135]:
from letor import LETOR
import pandas as pd
ranker_path = "lgbr_base.txt"
model_path = "lsi_base.model"
data_path = r'pickle\test_data.pkl'
qrel_path = r"qrels-folder\test_qrels.txt"

letor = LETOR(ranker_path, data_path)
qrels = letor.data_preparator.read_qrels_test(qrel_path)

data = letor.data

No sentence-transformers model found with name C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Object loaded from pickle\test_data.pkl


In [3]:
from letor import LETOR
import pandas as pd
ranker_path = "lgbr_base.txt"
model_path = "lsi_base.model"
data_path = r'pickle\test_data.pkl'
qrel_path = r"qrels-folder\test_qrels.txt"

letor = LETOR(ranker_path, model_path, data_path)
qrels = letor.data_preparator.read_qrels_test(qrel_path)

data = letor.data

ndcgs_letor = []
ndcgs_bsbi = []

columns = ['query_id', 'doc_id', 'score', 'relevance']
df_letor = pd.DataFrame(columns=columns)
df_bsbi = pd.DataFrame(columns=columns)

No sentence-transformers model found with name C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Object loaded from pickle\test_data.pkl


In [ ]:
from tqdm import tqdm
# iterate untuk setiap query
for qid in tqdm(qrels):
    dataset, doc_ids, raw_scores = data[qid]
    qrel = qrels[qid]
    k = 25

    # melakukan preparation data test dan prediksi 
    X = letor.ranker.prepare_data_test(dataset)
    pred = letor.ranker.predict_ranker(X)
    
    # untuk mendapatkan top k dokumen
    doc_pred_pairs = list(zip(doc_ids, pred))
    sorted_pairs = sorted(doc_pred_pairs, key=lambda x: x[1], reverse=True)
    top_k_doc_ids_letor = [pair[0] for pair in sorted_pairs[:k]]
    top_k_doc_ids_letor = ['D'+ doc for doc in top_k_doc_ids_letor]

    top_k_doc_ids_bsbi = doc_ids[:k]
    top_k_doc_ids_bsbi = ['D'+ doc for doc in top_k_doc_ids_bsbi]

    # mendapatkan ranking dari top k doc yang di sort
    ranking_letor = [qrel[did] for did in top_k_doc_ids_letor]
    ranking_bsbi = [qrel[did] for did in top_k_doc_ids_bsbi]
    
    # Hasil nDCG untuk model "letor" dan model "bsbi" akan dihitung dan disimpan.
    ndcg_score_letor = LETOR.ndcg(ranking_letor)
    ndcg_score_bsbi = LETOR.ndcg(ranking_bsbi)

    ndcgs_letor.append(ndcg_score_letor)
    ndcgs_bsbi.append(ndcg_score_bsbi)
    
    # Create DataFrames for 'letor' and 'bsbi' scores in the current loop
    df_letor_loop = pd.DataFrame({'query_id': [qid]*len(ranking_letor),
                                'doc_id': top_k_doc_ids_letor,
                                'score': [pair[1] for pair in sorted_pairs[:k]],
                                'relevance': ranking_letor})
    
    df_bsbi_loop = pd.DataFrame({'query_id': [qid]*len(ranking_letor),
                                'doc_id': top_k_doc_ids_bsbi,
                                'score': raw_scores[:k],
                                'relevance': ranking_bsbi})

    # Concatenate the DataFrames to the main DataFrames
    df_letor = pd.concat([df_letor, df_letor_loop], ignore_index=True)
    df_bsbi = pd.concat([df_bsbi, df_bsbi_loop], ignore_index=True)

print("Mean NDCG Score for Letor:", sum(ndcgs_letor) / len(ndcgs_letor))
print("Mean NDCG Score for BSBI:", sum(ndcgs_bsbi) / len(ndcgs_bsbi))

In [4]:
from tqdm import tqdm
# iterate untuk setiap query
for qid in tqdm(qrels):
    dataset, doc_ids, raw_scores = data[qid]
    qrel = qrels[qid]
    k = 25

    # melakukan preparation data test dan prediksi 
    X = letor.ranker.prepare_data_test(dataset)
    pred = letor.ranker.predict_ranker(X)
    
    # untuk mendapatkan top k dokumen
    doc_pred_pairs = list(zip(doc_ids, pred))
    sorted_pairs = sorted(doc_pred_pairs, key=lambda x: x[1], reverse=True)
    sorted_pairs = [('D'+doc, score) for doc, score in sorted_pairs]

    top_k_doc_ids_bsbi = doc_ids[:k]
    top_k_doc_ids_bsbi = ['D'+ doc for doc in top_k_doc_ids_bsbi]
    
    # mendapatkan ranking dari top k doc yang di sort
    list_rel = []
    curr_score = 0
    rel = 0
    for did,score in sorted_pairs:
        if len(list_rel)==k+1:
            break
        if score != curr_score:
            list_rel.append(rel)
            rel = 0
            curr_score = score
            if qrel[did]: rel=1
        else:
            if not rel: rel=qrel[did]  

    ranking_letor = list_rel[1:]
    ranking_bsbi = [qrel[did] for did in top_k_doc_ids_bsbi]
    
    # Hasil nDCG untuk model "letor" dan model "bsbi" akan dihitung dan disimpan.
    ndcg_score_letor = LETOR.ndcg(ranking_letor)
    ndcg_score_bsbi = LETOR.ndcg(ranking_bsbi)

    ndcgs_letor.append(ndcg_score_letor)
    ndcgs_bsbi.append(ndcg_score_bsbi)

print("Mean NDCG Score for Letor:", sum(ndcgs_letor) / len(ndcgs_letor))
print("Mean NDCG Score for BSBI:", sum(ndcgs_bsbi) / len(ndcgs_bsbi))

100%|██████████| 700/700 [03:56<00:00,  2.96it/s]

Mean NDCG Score for Letor: 0.4983136725250903
Mean NDCG Score for BSBI: 0.6134689162548457


In [45]:
# Save df_letor to a CSV file
df_letor.to_csv('df_letor.csv', index=False)

# Save df_bsbi to a CSV file
df_bsbi.to_csv('df_bsbi.csv', index=False)

## BERT

In [2]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("m-aliabbas1/tiny_bert_29_medicare_intents")

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/712k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

No sentence-transformers model found with name C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:

embeddings = sentence_model.encode(["haloo nama gw bryan tjandra, lu siapa?"], show_progress_bar=True)

from sklearn.preprocessing import normalize
normalized_embeddings = normalize(embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
normalized_embeddings.flatten().shape

(128,)

In [10]:
from ranker import Ranker
from data_preparator import DataPreparator

train_doc_pkl = r'pickle\train_documents.pkl'
train_query_pkl = r'pickle\train_queries.pkl'
val_doc_pkl = r'pickle\val_documents.pkl'
val_query_pkl = r'pickle\val_queries.pkl'

# load file dari pickle
train_documents = DataPreparator.load_from_pickle(train_doc_pkl)
train_queries = DataPreparator.load_from_pickle(train_query_pkl)
val_documents = DataPreparator.load_from_pickle(val_doc_pkl)
val_queries = DataPreparator.load_from_pickle(val_query_pkl)

# membuat model LSI dari dokumen train
ranker = Ranker()
# ranker.create_lsi_model(train_documents)

# mempersiapkan train dataset dan validation dataset
train_qrel_path = r"qrels-folder\train_qrels.txt"
val_qrel_path = r"qrels-folder\val_qrels.txt"

Object loaded from pickle\train_documents.pkl
Object loaded from pickle\train_queries.pkl
Object loaded from pickle\val_documents.pkl
Object loaded from pickle\val_queries.pkl


No sentence-transformers model found with name C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\bryan/.cache\torch\sentence_transformers\m-aliabbas1_tiny_bert_29_medicare_intents were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
train_dataset, train_group_qid_count = ranker.data_preparator.prepare_training_dataset(train_qrel_path, train_queries, train_documents)
val_dataset, val_group_qid_count = ranker.data_preparator.prepare_training_dataset(val_qrel_path, val_queries, val_documents)

In [66]:
from scipy.spatial.distance import cosine
from tqdm import tqdm

def features(query, doc, is_batch=False):
    """
    Fungsi untuk membangun fitur untuk training model
    X: fitur, Y: target
    """

    if not is_batch:
        # Hasil BERT
        v_q = normalize(sentence_model.encode([query])).flatten().tolist()
        v_d = normalize(sentence_model.encode([doc])).flatten().tolist()
        
        # Fitur tambahan cosine dan jaccard
        cosine_dist = cosine(v_q, v_d)
        jaccard = len(set(query.split()) & set(doc.split())) / len(set(query.split()) | set(doc.split()))
        return v_q + v_d + [jaccard, cosine_dist]
    else:
        # Handle batch queries and documents
        # print(query, doc)
        batch_features = []
        v_q = normalize(sentence_model.encode(query)).tolist()
        v_d = normalize(sentence_model.encode(doc)).tolist()
            
        for i, (query, doc) in enumerate(zip(query, doc)):

            cosine_dist = cosine(v_q[i], v_d[i])
            jaccard = len(set(query.split()) & set(doc.split())) / len(set(query.split()) | set(doc.split()))
            
            batch_features.append(v_q[i] + v_d[i] + [jaccard, cosine_dist])
        return batch_features

In [67]:
batch_size = 256  # Specify the batch size
is_batch = batch_size > 1

X = []
Y = []
batch_queries = []
batch_docs = []

for data in tqdm(train_dataset):
    query, doc, rel = data
    query = ' '.join(query)
    doc = ' '.join(doc)
    
    batch_queries.append(query)
    batch_docs.append(doc)
    Y.append(rel)
    
    if len(batch_queries) == batch_size:
        if is_batch:
            X.extend(features(batch_queries, batch_docs, is_batch))
        else:
            X.append(features(batch_queries, batch_docs, is_batch))
        
        # Clear the batch lists
        batch_queries = []
        batch_docs = []

# Process any remaining data in the last batch
if batch_queries:
    if is_batch:
        X.extend(features(batch_queries, batch_docs, is_batch))
    else:
        X.append(features(batch_queries, batch_docs, is_batch))

100%|██████████| 285600/285600 [07:59<00:00, 595.41it/s]


In [73]:
import numpy as np
X = np.array(X)
X.shape

(285600, 258)

In [74]:
Y = np.array(Y)
Y.shape

(285600,)

In [ ]:


# membuat representasi fitur X dan label Y
X_train, Y_train = ranker.prepare_data_train(train_dataset)
X_val, Y_val = ranker.prepare_data_train(val_dataset)

# melatih model ranker
ranker.fit_ranker(X_train, Y_train, train_group_qid_count,
                    X_val, Y_val, val_group_qid_count)

# Setelah melakukan fit pada model
best_score = ranker.ranker.best_score_['val'] 
print(f"Best NDCG score on validation set: {best_score}")
